In [1]:
import frontmatter
import re
import urllib
import requests

DIRNAME = os.path.abspath('')
WWW_SHIPS_DATA_PATH = os.path.join(DIRNAME, "../../www/views/ships")
WWW_IMG_PATH = os.path.join(DIRNAME, "../../www/assets/img")

In [2]:
for ship_filename in os.listdir(WWW_SHIPS_DATA_PATH):
    md_match = re.match(r"(.*)\.md", ship_filename)
    if not md_match:
        continue
    ship_slug = md_match.groups()[0]
    ship_frontmatter = frontmatter.load(os.path.join(WWW_SHIPS_DATA_PATH, ship_filename))
    wikipedia_url = ship_frontmatter.metadata.get("wikipediaUrl")
    photo_path = ship_frontmatter.metadata.get("photo", "")
    if photo_path != "" or not wikipedia_url:
        continue
    parsed = urllib.parse.urlparse(wikipedia_url)
    country, title = re.match(r".*([a-z]{2,3})\.wikipedia\.org\/wiki\/(.*)", wikipedia_url).groups()
    json_url = f"http://{country}.wikipedia.org/w/api.php?action=query&titles={title}&prop=pageimages&format=json&pithumbsize=500"
    data = requests.get(json_url).json()
    keys = list(data["query"]["pages"])
    image_url = data["query"]["pages"][keys[0]].get("thumbnail", {}).get("source")
    if not image_url:
        print(f"could not find image url in {json_url}")
        continue
    img_ext = image_url[-10:].split(".")[-1]
    img_filename = f"{ship_slug}.{img_ext}"
    with open(os.path.join(f"{WWW_IMG_PATH}", img_filename), "wb") as f:
        f.write(requests.get(image_url).content)
    print(f"wrote image www/assets/js/{img_filename}")
    ship_frontmatter.metadata["photo"] = f"/img/{img_filename}"
    with open(os.path.join(WWW_SHIPS_DATA_PATH, ship_filename), "w") as f:
        f.write(frontmatter.dumps(ship_frontmatter))
    print(f"wrote back md to www/views/ships/{ship_filename}")
        

wrote image www/assets/js/caribbean-princess-9215490.jpg
wrote back md to www/views/ships/caribbean-princess-9215490.md
wrote image www/assets/js/star-breeze-8807997.jpg
wrote back md to www/views/ships/star-breeze-8807997.md
wrote image www/assets/js/veendam-9102992.jpg
wrote back md to www/views/ships/veendam-9102992.md
wrote image www/assets/js/zuiderdam-9221279.JPG
wrote back md to www/views/ships/zuiderdam-9221279.md
could not find image url in http://en.wikipedia.org/w/api.php?action=query&titles=MV_Nova_Star&prop=pageimages&format=json&pithumbsize=500
wrote image www/assets/js/amsterdam-9188037.JPG
wrote back md to www/views/ships/amsterdam-9188037.md
wrote image www/assets/js/star-pride-8707343.JPG
wrote back md to www/views/ships/star-pride-8707343.md
wrote image www/assets/js/pride-of-york-8501957.jpg
wrote back md to www/views/ships/pride-of-york-8501957.md
wrote image www/assets/js/spirit-of-france-9533816.JPEG
wrote back md to www/views/ships/spirit-of-france-9533816.md
wr